# 导入工具包

In [1]:
# opencv-python
import cv2

# mediapipe人工智能工具包
import mediapipe as mp

# 进度条库
from tqdm import tqdm

# 时间库
import time

# 导入模型

In [2]:
# 导入三维人脸关键点检测模型
mp_face_mesh = mp.solutions.face_mesh
model = mp_face_mesh.FaceMesh(   
        static_image_mode=False,      # 是静态图片还是连续视频帧，摄像头画面为连续视频帧，此处选False
        refine_landmarks=True,       # 使用Attention Mesh Model，对嘴唇、眼睛、瞳孔周围的关键点精细定位
        max_num_faces=5,              # 最多检测几张脸
        min_detection_confidence=0.5, # 置信度阈值
        min_tracking_confidence=0.5,  # 追踪阈值
)

# 导入可视化函数和可视化样式
mp_drawing = mp.solutions.drawing_utils 
# 关键点可视化样式
landmark_drawing_spec = mp_drawing.DrawingSpec(thickness=1, circle_radius=2, color=[66,77,229])
# 轮廓可视化样式
connection_drawing_spec = mp_drawing.DrawingSpec(thickness=2, circle_radius=1, color=[223,155,6])

# 处理单帧的函数

In [3]:
# 处理帧函数
def process_frame(img):
    
    # 记录该帧开始处理的时间
    start_time = time.time()
    scaler = 1 # 文字大小
    
    # 获取图像宽高
    h,w = img.shape[0], img.shape[1]
    # BGR转RGB
    img_RGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    # 将RGB图像输入模型，获取预测结果
    results = model.process(img_RGB)

    if results.multi_face_landmarks: # 如果检测出人脸
        
        for face_landmarks in results.multi_face_landmarks: # 遍历每一张脸
            mp_drawing.draw_landmarks(
                  image=img,
                  landmark_list=face_landmarks,
                  # connections=mp_face_mesh.FACEMESH_TESSELATION, # 可视化Face Mesh
                  connections=mp_face_mesh.FACEMESH_CONTOURS,  # 可视化脸轮廓
                  landmark_drawing_spec=landmark_drawing_spec, # 关键点圆圈样式
                  connection_drawing_spec=connection_drawing_spec) # 轮廓样式
            for idx, coord in enumerate(face_landmarks.landmark): # 遍历每一个关键点
                cx = int(coord.x * w)
                cy = int(coord.y * h)
                # 图片，添加的文字，左上角坐标，字体，字体大小，颜色，字体粗细
                img = cv2.putText(img, 'Face Detected', (25 * scaler, 50 * scaler), cv2.FONT_HERSHEY_SIMPLEX, 1.25 * scaler, (255, 0, 255), 2 * scaler)
                img = cv2.putText(img, str(idx), (cx, cy), cv2.FONT_HERSHEY_SIMPLEX, 0.3*scaler, (0, 255, 0), 1)
    else:
        img = cv2.putText(img, 'No Face Detected', (25 * scaler, 50 * scaler), cv2.FONT_HERSHEY_SIMPLEX, 1.25 * scaler, (255, 0, 255), 2 * scaler)
    
    # 记录该帧处理完毕的时间
    end_time = time.time()
    # 计算每秒处理图像帧数FPS
    FPS = 1/(end_time - start_time)  
    
    img = cv2.putText(img, 'FPS  '+str(int(FPS)), (25 * scaler, 100 * scaler), cv2.FONT_HERSHEY_SIMPLEX, 1.25 * scaler, (255, 0, 255), 2 * scaler)
    return img

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


# 调用摄像头获取每帧（模板）

In [4]:
# 调用摄像头逐帧实时处理模板
# 不需修改任何代码，只需修改process_frame函数即可

# 导入opencv-python
import cv2
import time

# 获取摄像头，传入0表示获取系统默认摄像头
cap = cv2.VideoCapture(1)

# 打开cap
cap.open(0)

# 无限循环，直到break被触发
while cap.isOpened():
    # 获取画面
    success, frame = cap.read()
    if not success:
        print('Error')
        break
    start_time = time.time()
    
    ## !!!处理帧函数
    frame = process_frame(frame)
    
    # 展示处理后的三通道图像
    cv2.imshow('my_window',frame)

    if cv2.waitKey(1) in [ord('q'),27]: # 按键盘上的q或esc退出（在英文输入法下）
        break
    
# 关闭摄像头
cap.release()

# 关闭图像窗口
cv2.destroyAllWindows()

[ WARN:0@0.573] global /io/opencv/modules/videoio/src/cap_v4l.cpp (902) open VIDEOIO(V4L2:/dev/video1): can't open camera by index


# 视频逐帧处理（模板）

In [8]:
# 视频逐帧处理代码模板
# 不需修改任何代码，只需定义process_frame函数即可

def generate_video(input_path='./videos/three-hands.mp4'):
    filehead = input_path.split('/')[-1]
    output_path = "out-" + filehead
    
    print('视频开始处理',input_path)
    
    # 获取视频总帧数
    cap = cv2.VideoCapture(input_path)
    frame_count = 0
    while(cap.isOpened()):
        success, frame = cap.read()
        frame_count += 1
        if not success:
            break
    cap.release()
    print('视频总帧数为',frame_count)
    
    # cv2.namedWindow('Crack Detection and Measurement Video Processing')
    cap = cv2.VideoCapture(input_path)
    frame_size = (cap.get(cv2.CAP_PROP_FRAME_WIDTH), cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # fourcc = int(cap.get(cv2.CAP_PROP_FOURCC))
    # fourcc = cv2.VideoWriter_fourcc(*'XVID')
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    fps = cap.get(cv2.CAP_PROP_FPS)

    out = cv2.VideoWriter(output_path, fourcc, fps, (int(frame_size[0]), int(frame_size[1])))
    
    # 进度条绑定视频总帧数
    with tqdm(total=frame_count-1) as pbar:
        try:
            while(cap.isOpened()):
                success, frame = cap.read()
                if not success:
                    break

                # 处理帧
                # frame_path = './temp_frame.png'
                # cv2.imwrite(frame_path, frame)
                try:
                    frame = process_frame(frame)
                except:
                    print('error')
                    pass
                
                if success == True:
                    # cv2.imshow('Video Processing', frame)
                    out.write(frame)

                    # 进度条更新一帧
                    pbar.update(1)

                # if cv2.waitKey(1) & 0xFF == ord('q'):
                    # break
        except:
            print('中途中断')
            pass

    cv2.destroyAllWindows()
    out.release()
    cap.release()
    print('视频已保存', output_path)

In [9]:
generate_video(input_path='videos/single_person2.mp4')

视频开始处理 videos/single_person2.mp4
视频总帧数为 0


0it [00:00, ?it/s]

视频已保存 out-single_person2.mp4


如果遇到Error，需检查其它notebook是否占用了摄像头，需在其它notebook中restart kernel